In [1]:
import os
import selenium
from selenium import webdriver
import time
from PIL import Image
import io
import requests
# from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException
import time     
import os        
import bs4
from bs4 import BeautifulSoup 
import scipy as sc
import numpy as np
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re

In [2]:
fix_url = 'https://www.tripadvisor.com'
url ='https://www.tripadvisor.com/Attractions-g187768-Activities-oa0-Italy.html'

In [3]:
# driver = webdriver.Chrome('chromedriver')

In [4]:
driver = webdriver.Chrome('chromedriver')

In [5]:
# recives the url(string)
# returns the html code of the url in Beutifulsoup obj(html.parser)

def Load_the_code(url):
    driver.get(url)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source,'html.parser')
    return soup

In [6]:
# recives soup object of trip advisor(first) page 
# returns the num(int) of pages we need to scrap
def Find_how_much_results(soup1):
    s = soup1.find('div',class_="Ci")
    test_string = s.text
    test_string = test_string.replace(',','')
    arr = re.findall(r'\d+', test_string)
    arr = [int(i) for i in arr]
    num = max(arr)
    if num <=30:
        return 1;
    else:
        return (num//30) +1

In [7]:
# recives objective of soup and extracts the "next page" link
# returns the link(string)
def Find_the_next_link(soup2):
    obj = soup2.find_all("a",class_="BrOJk u j z _F wSSLS tIqAi unMkR")
    if obj[0]['aria-label'] == 'Previous page':
        return obj[1]['href']
    else:
        return obj[0]['href']

In [8]:
# get the list of all attractions from the url:

def get_list_of_attrs_links_from_page(soup_obj):
    list_of_atters_links=[]
    eror_count = 0
    all_attrractions_raw_code_list = soup_obj.find_all("div",class_="alPVI eNNhq PgLKC tnGGX")
    for attr in all_attrractions_raw_code_list:
        try:
            l=attr.find("a")
            list_of_atters_links.append("https://www.tripadvisor.com/"+l['href'])
        except:
            eror_count+=1
            continue
        
    return list_of_atters_links,eror_count

In [9]:
# extracts the name of the attraction

def Extract_name_of_attraction(soup_obj):
    try:
        res=soup_obj.find("h1",class_="biGQs _P fiohW eIegw")
        res = res.string
        return res
    except:
        return np.nan
    

In [10]:
def Extract_tag_1_and_tag_2(soup_obj):
    try:
        name = soup_obj.find("div",class_="fIrGe _T bgMZj")
        st=name.string
        return st
        try:
            x =re.findall(r"\w+[\s\w&]+",st)
            if len(x) >1:
                return x[0],x[1]
            else:
                return x[0],np.nan
                
        except:
            return np.nan,np.nan
    except:
            return np.nan

In [11]:
def Extract_num_of_reviewers(soup_obj):
    try:
        name = soup_obj.find("span",class_="yyzcQ")
        temp_string=name.string
        temp_string =temp_string.replace(",","")
        temp_int = int(temp_string)
        return temp_int
    except:
        return np.nan
    
    

In [12]:
def Extract_Rank_score(soup_obj):
    try:
        rank = soup_obj.find("div",class_='biGQs _P fiohW hzzSG uuBRH')
        rank = rank.string
        rank = float(rank)
        return rank
    except:
        return np.nan

In [13]:
def Extract_num_of_resturants_nearby(soup_obj):
    try:
        num_of_rests = soup_obj.find("div",class_='biGQs _P pZUbB mowmC KxBGd')
        num_of_rests = num_of_rests.string
        num_of_rests = re.search(r"\d[\W\d]+",num_of_rests)
        num_of_rests = num_of_rests.group(0)
        num_of_rests = num_of_rests.replace(r",","")
        return num_of_rests
    except:
        return np.nan


In [14]:
def Extract_num_of_attractions_nearby(soup_obj):
     try:
        num_of_att = soup_obj.find_all("div",class_='biGQs _P pZUbB mowmC KxBGd')
        num_of_att = num_of_att[1].string
        num_of_att = re.search(r"\d[\W\d]+",num_of_att)
        num_of_att = num_of_att.group(0)
        num_of_att = num_of_att.replace(",","")
        num_of_att = num_of_att.replace(" ","")
        return num_of_att
     except:
        return np.nan

    

In [15]:
def Extract_destric(soup_obj):
    try:
        f = soup_obj.find_all("span",class_='biGQs _P XWJSj Wb')
        f = [i.string for i in f]
        for i in f:
            j = re.search("Italy",i)
            if j != None:
                j=i
                break
        return j
    except:
        return np.nan

In [16]:
def Extract_visit_time(soup_obj):
     try:
        time = soup_obj.find("div",class_='_c')
        time = time.string
        return time
     except:
        return np.nan
    

In [17]:
def Extract_opening_houres(soup_obj):
    try:
        opening_houres = soup_obj.find("span",class_='EFKKt')
        opening_houres = opening_houres.string
        return opening_houres
    except:
        return np.nan
    
    

In [18]:
df_of_primary_urls = pd.read_csv('new_1500')
df_of_primary_urls = df_of_primary_urls['page_url']
df_of_primary_urls = df_of_primary_urls.iloc[50:100]

In [47]:
site_name=[]
tag_1=[]
# tag_2=[]
total_reviewers=[]
rank_list = []
num_of_resturants_nearby=[]
num_of_attractions_nearby=[]
opening_houres =[]
recomended_amount_time_to_visit=[]
distric_city=[]



df_of_primary_urls = pd.read_csv('new_1500')
df_of_primary_urls = df_of_primary_urls['page_url']
df_of_primary_urls = df_of_primary_urls.iloc[900:900]


for ur in df_of_primary_urls:
    try:
        driver.get(ur)
        code = driver.page_source
        soup = BeautifulSoup(code,'html.parser')
        time.sleep(5)
        list_of_att_url,eror_count = get_list_of_attrs_links_from_page(soup)
    except:
        driver = webdriver.Chrome('chromedriver')
        driver.get(ur)
        code = driver.page_source
        time.sleep(5)
        soup = BeautifulSoup(code,'html.parser')
        list_of_att_url,eror_count = get_list_of_attrs_links_from_page(soup)
        
        

# ### ------ this part is for extracting only one page of results---------------


    for url in list_of_att_url:
        try:

            try:
                driver.get(url)
                time.sleep(5)
            except:
                driver = webdriver.Chrome('chromedriver')
                time.sleep(5)
                continue

            try:
                soup = driver.page_source
                obj = BeautifulSoup(soup,'html.parser')
            except:
                print(f" stoped at {url}")
                break
            name = Extract_name_of_attraction(obj) 
            tag1= Extract_tag_1_and_tag_2(obj)
            num_of_rev=Extract_num_of_reviewers(obj)
            rank=Extract_Rank_score(obj)
            rest=Extract_num_of_resturants_nearby(obj)
            att = Extract_num_of_attractions_nearby(obj)
            dest = Extract_destric(obj)
            opening = Extract_opening_houres(obj)
            visit_time = Extract_visit_time(obj)

            tag_1.append(tag1)    
            site_name.append(name)
            total_reviewers.append(num_of_rev)
            rank_list.append(rank)
            num_of_resturants_nearby.append(rest)
            num_of_attractions_nearby.append(att)
            recomended_amount_time_to_visit.append(visit_time)
            distric_city.append(dest)
            opening_houres.append(opening)

        except:
            print(f" stooped at {url}")
    ### --------------------------------------------------------------------------  

In [48]:
d = {"site_name":site_name,"tag_1":tag_1,"distric_city":distric_city,"total_reviewers":total_reviewers,
     "rank":rank_list,"num_of_resturants_nearby":num_of_resturants_nearby,
     "num_of_attractions_nearby":num_of_attractions_nearby
     ,"recomended_amount_time_to_visit":recomended_amount_time_to_visit
     ,"opening_houres":opening_houres}

curr_iteration_30_res = pd.DataFrame(d)
curr_iteration_30_res.to_csv('first_900_950')

